In [69]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin) :    
    def __init__(self, columns):
        self.columns=columns
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.columns].values


In [175]:
X=np.array(['Master Jimmy', 'Mrs. Smith', 'Mr Smith'])
f1 = (np.char.find(X[:], 'Master')!=-1) # which entries contain the f key
X[f1]=1
X

array(['1', 'Mrs. Smith', 'Mr Smith'], dtype='<U12')

In [201]:
import numpy.testing as test

class NameEncoder(BaseEstimator, TransformerMixin) :    
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        flop=['Master', 'Miss', 'Mrs.', 'Mr.']
        f_b = np.zeros(X.size, dtype=bool) # initialize as false
        print(X
        for f in flop:
            f1 = (np.char.find(X[:,0], f)!=-1) # which entries contain the f key
            X[f1]=flop.index(f) # replace with the actual index in the array string            
            f_b = f_b | f1 # retain the matched ones
        X[~f_b]=4 # undefined
        return X.astype(np.int)
    @staticmethod
    def test():
        nameEncoder = NameEncoder()
        res = nameEncoder.fit_transform(np.array([['Master Jimmy'], ['Mrs. Smith'], ['Mr. Smith']]))
        test.assert_array_equal(res, [[0],[2],[3]])
        
        
NameEncoder.test()


SyntaxError: invalid syntax (<ipython-input-201-902833aa5387>, line 10)

In [136]:
# age imputer expects an 2D array having on the first column
# the encoded name 0,1,2,3,4 and on the second column the age
class AgeImputer(BaseEstimator, TransformerMixin) :        
    # fit will calculate the averages per encoded name
    def fit(self, X, y=None):
        self.titleAgeDictionary = {}
        for key in np.arange(0, 5):
            try: # just to make sure for unit tests add a try/catch
                self.titleAgeDictionary[key] = np.median(x[x[:,0] == key][1])
            except:
                pass
        return self

    def transform(self, X, y=None):        
        for key in titleAgeDictionary.keys():
            X[(X[:,0] == key) & (np.isnan(X[:,1])),1]=titleAgeDictionary[key]
        return X
        

In [199]:
#test
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline

%matplotlib inline

# read the dataframe
df = pd.read_csv('train.csv')
f = lambda x : 0 if ( x != x) else 1;
df['Cabin'] = df['Cabin'].apply(f)

dfSelector = DataFrameSelector(['Name'])
r = dfSelector.fit_transform(df)

pipelineName = Pipeline([
                ('selector', DataFrameSelector(['Name'])),
                ('name_encoder', NameEncoder())])
x = pipelineName.fit_transform(df)



TypeError: string operation on non-string array

array([[1. , 1. ],
       [1. , 2. ],
       [2. , 4. ],
       [2. , 5. ],
       [2. , 6. ],
       [3. , 5. ],
       [2. , 3.5],
       [2. , 3.5]])